In [ ]:
"""copy paste this into 3d slicer python interactor 
to extract binary labelmaps from a segmentation with
multiple labels with associated names"""

In [ ]:
import os
import glob
import slicer
import vtk

def process_segmentation(file_path, output_folder, segment_names):
    # Load segmentation
    segmentationNode = slicer.util.loadSegmentation(file_path)
    if not segmentationNode:
        return  # Skip file if it fails to load

    # Prepare to export only specified segments
    segmentation = segmentationNode.GetSegmentation()
    segmentIds = vtk.vtkStringArray()
    for segmentName in segment_names:
        segmentId = segmentation.GetSegmentIdBySegmentName(segmentName)
        if segmentId:
            segmentIds.InsertNextValue(segmentId)

    # Create a new label map node
    labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")

    # Reference volume can be set here if necessary
    referenceVolumeNode = None  # Assuming no reference volume is required, otherwise reference it here

    # Export specified segments to the label map node
    export_success = slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsToLabelmapNode(
        segmentationNode, segmentIds, labelmapVolumeNode, referenceVolumeNode)

    if export_success:
        # Save the label map as NIfTI
        output_path = os.path.join(output_folder, os.path.basename(file_path).replace('.nrrd', '.nii.gz'))
        save_success = slicer.util.saveNode(labelmapVolumeNode, output_path)
        if save_success:
            print(f"Output saved: {output_path}")

    # Clear the created nodes from the scene to free memory
    slicer.mrmlScene.RemoveNode(labelmapVolumeNode)
    slicer.mrmlScene.RemoveNode(segmentationNode)

segment_names = ["Lobar"]  # Segments to be processed, you can add a list here

# Directory setup
input_folder = r'E:\G-AI neuroimaging\test files\nrrd test'
output_folder = r'E:\G-AI neuroimaging\test files\test nifty'


# Ensure the output directory exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each .nrrd file in the input directory
for file_path in glob.glob(os.path.join(input_folder, '*.nrrd')):
    process_segmentation(file_path, output_folder, segment_names)


In [ ]:
#verbose version, for debug
import os
import glob
import slicer
import vtk

def process_segmentation(file_path, output_folder, segment_names):
    print(f"Loading segmentation from {file_path}")
    segmentationNode = slicer.util.loadSegmentation(file_path)
    
    if not segmentationNode:
        print(f"Failed to load segmentation from {file_path}")
        return

    # Preparing to export only specified segments
    segmentation = segmentationNode.GetSegmentation()
    segmentIds = vtk.vtkStringArray()
    for segmentName in segment_names:
        segmentId = segmentation.GetSegmentIdBySegmentName(segmentName)
        if segmentId:
            segmentIds.InsertNextValue(segmentId)
            print(f"Segment '{segmentName}' with ID '{segmentId}' is ready for export.")
        else:
            print(f"Segment '{segmentName}' not found in {file_path}")

    # Create a new label map node
    labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
    
    # Try to manually set a reference volume node if known, or leave as None if not necessary
    referenceVolumeNode = None  # Assume no reference volume if none is specifically required
    
    # If you know the reference volume, uncomment and adjust the following line:
    # referenceVolumeNode = slicer.util.getNode('NameOfYourReferenceVolumeNode')
    
    # Export specified segments to the label map node
    print("Exporting segments to labelmap node...")
    success = slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsToLabelmapNode(
        segmentationNode, segmentIds, labelmapVolumeNode, referenceVolumeNode)

    if success:
        print("Export successful.")
    else:
        print("Export failed.")
        return

    # Save the label map as NIfTI
    output_path = os.path.join(output_folder, os.path.basename(file_path).replace('.nrrd', '.nii.gz'))
    result = slicer.util.saveNode(labelmapVolumeNode, output_path)
    if result:
        print(f"Label map successfully saved to {output_path}")
    else:
        print(f"Failed to save label map to {output_path}")

# Directory setup and execution loop
input_folder = r'E:\G-AI neuroimaging\test files\nrrd test'
output_folder = r'E:\G-AI neuroimaging\test files\test nifty'
segment_names = ["Lobar"]  # Segments to be processed

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created output folder at {output_folder}")

for file_path in glob.glob(os.path.join(input_folder, '*.nrrd')):
    process_segmentation(file_path, output_folder, segment_names)
